In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import datetime as dt
import sqlalchemy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import matplotlib.dates as mdates

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
# reflect the tables
Base = automap_base()
Base.prepare(engine, reflect=True)
keys = Base.classes.keys()
print(f"Here are the keys to the kingdom")
print(*keys, sep=", ")

In [ ]:
# View all of the classes that automap found
Measurement= Base.classes.measurement
Station = Base.classes.station

In [ ]:
#Find out what are the columns names by calling the dictionary of class Measurement
session = Session(engine)
measure_first_row = session.query(Measurement).first()
measure_first_row.__dict__


# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
measurement = pd.DataFrame({"date":dates,
                             "precipitation":precip})
measurement.dropna(inplace = True)
measurement

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 
# Calculate the date one year from the last date in data set.
# Perform a query to retrieve the data and precipitation scores
# Save the query results as a Pandas DataFrame and set the index to the date column
# Sort the dataframe by date
# Use Pandas Plotting with Matplotlib to plot the data

lastdate = session.query(func.max(Measurement.date)).\
            scalar()

dt_lastdate= dt.datetime.strptime(lastdate,"%Y-%m-%d").date()
dt_startdate = dt_lastdate - dt.timedelta(days=365)
startdate = dt_startdate.strftime("%Y-%m-%d")

query = session.query(Measurement).\
            filter(Measurement.date.between(startdate,lastdate)).\
            all()

session.close()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
dates = []
precip = []
for row in query:
    measurement = row
    dates.append(measurement.date)
    precip.append(measurement.prcp)

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
measurement = pd.DataFrame({"date":dates,
                             "precipitation":precip})
measurement.dropna(inplace = True)
measurement

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
plt.style.use('fivethirtyeight')

date = measurement.index
MDate = mdates.datestr2num(date)
precipitation = measurement['precipitation']

#Setting the date form
months_fmt = mdates.DateFormatter('%M')

#the function subplots
fig, ax = plt.subplots()

ax.plot(MDate,precipitation,label = 'Precipitation')

#setting the location of the format of the x-axis ticks.
ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
fig.autofmt_xdate()

#the limits of the x an y axes.
datemin = np.datetime64(date[0])
datemax = np.datetime64(date[-1])
precipmin = precipitation.min()
precipmax = precipitation.max() + .25
ax.set_xlim(datemin,datemax)
ax.set_ylim(precipmin,precipmax)
ax.grid(True)
ax.legend(loc=9)

plt.show()


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
top_station = session.query(Measurement.station).\
                group_by(Measurement.station).\
                order_by(func.count(Measurement.station).desc()).\
                first()
t_station = top_station.station


top_station_stats = session.query(Measurement, 
                                  func.max(Measurement.tobs).label('max_temp'),
                                  func.min(Measurement.tobs).label('min_temp'),
                                  func.avg(Measurement.tobs).label('avg_temp'),
                                  Station.name.label('sta_name')).\
                    join(Station, Measurement.station==Station.station).\
                    filter(Measurement.station == t_station).\
                    first()

print(f'{top_station_stats.sta_name} had a maximum temperature of {top_station_stats.max_temp}')
print(f'{top_station_stats.sta_name} had a minimum temperature of {top_station_stats.min_temp}')
print(f'{top_station_stats.sta_name} had a average

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
subquery = session.query(Station.station).filter(Station.name.like('%WAIHEE%')).subquery()
waihee_last_year = session.query(Measurement).\
                   filter(Measurement.station.in_(subquery)).\
                   filter(Measurement.date.between(startdate,lastdate))
session.close()


In [ ]:
waihee_tobs = []
for row in waihee_last_year:
    waihee_tobs.append(row.tobs)

In [ ]:
fig,ax = plt.subplots(tight_layout=True, sharey=True)

ax.hist(waihee_tobs,bins=12,label='tobs', color='#a6cee3')


ax.set_xlabel('Temperature')
ax.set_ylabel('No. of Instances')
ax.set_title('No. of Instances of Temperature at Waihee')

ax.legend()

#Saving the image in png format
plt.savefig('results/waihee-histogram.png', dbi=300, bbox_inches='tight')

plt.show()

# Close session

In [ ]:
# Close Session
session.close()